# Trabajo de Inteligencia artificial
 ## Análisis de noticias

 Realizado por:
 - Marta Aguilar Morcillo
 - Candela Jazmín Gutiérrez González

Fecha: 30/05/2025

Convocatoria de junio.

 ## 1. Lectura de datos

 Se comenzará con la lectura del corpus. Para ello, será necesaria la importación de las siguientes librerías:
 - **nltk:** 
 - **punkt_tab:** para la tokenización de las palabras de los documentos.
 - **contractions:**
 - **sklearn:**

In [1]:
!pip install nltk
import nltk

from nltk import download

download('punkt_tab')                           # Tokenización
nltk.download('averaged_perceptron_tagger')     # POS tagging
nltk.download('averaged_perceptron_tagger_eng') # POS tagging
nltk.download('wordnet')                        # WordNet lemmatizer
nltk.download('omw-1.4')                        # WordNet multilingüe

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.data import path
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from nltk.corpus import wordnet as wnimport numpy as 

path.append(".")

SyntaxError: invalid syntax (1342639323.py, line 9)

In [ ]:
!pip install contractions
import contractions

In [ ]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
import re
from bs4 import MarkupResemblesLocatorWarning
import warnings

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
import spacy

In [ ]:
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

In [ ]:
palabras_vacias_ingles = stopwords.words('english')

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def elimina_html(contenido):
    return BeautifulSoup(contenido).get_text()

def elimina_no_alfanumerico(contenido):
    return [re.sub(r'[^\w]', '', palabra)
            for palabra in contenido
            if re.search(r'\w', palabra)]

def expandir_constracciones(contenido):
    return contractions.fix(contenido)

def pasar_a_minuscula(contenido):
    return contenido.lower()

def limpiar_texto(texto):
    texto = re.sub(r'[^a-zA-Z\s]', ' ', texto)  # Reemplaza todo lo que no es letra o espacio con espacio
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def elimina_palabras_vacias(contenido):
    return [palabra for palabra in contenido if palabra not in palabras_vacias_ingles]

def lematizador(contenido):
    lemmatizer = WordNetLemmatizer()
    pos_tags = pos_tag(contenido)

    resultado = []
    for palabra, tag in pos_tags:
        if tag.startswith('VB'):  # Verbos
            resultado.append(lemmatizer.lemmatize(palabra, pos='v'))  # infinitivo
        else:  # Sustantivos y el resto tal como están
            resultado.append(palabra)

    return resultado

def extraer_noun_chunks(tokens):
    resultados = []
    doc = nlp(" ".join(tokens))
    
    noun_chunks = [chunk.text.lower().strip() for chunk in doc.noun_chunks if len(chunk.text.split()) <= 3]
    noun_chunks_set = set(noun_chunks)

    i = 0
    while i < len(tokens):
        composed2 = " ".join(tokens[i:i+2]).lower()
        composed3 = " ".join(tokens[i:i+3]).lower()

        if composed3 in noun_chunks_set:
            i += 3  
        elif composed2 in noun_chunks_set:
            i += 2 
        else:
            resultados.append(tokens[i].lower())  
            i += 1

    return resultados + noun_chunks


In [ ]:
def proceso_contenido(texto):
    texto = elimina_html(texto)
    texto = expandir_constracciones(texto)
    texto = pasar_a_minuscula(texto)
    texto = limpiar_texto(texto)                # Limpiar antes de tokenizar
    tokens = word_tokenize(texto)               
    tokens = elimina_no_alfanumerico(tokens)    # Limpiar tokens individuales
    tokens = elimina_palabras_vacias(tokens)
    tokens = lematizador(tokens)
    return tokens

In [ ]:
def lectura_normalizada_corpus():
    df = pd.read_csv("news_corpus.csv", encoding="latin-1", sep=";", quotechar='"')
    resultados = []

    for index, fila in df.iterrows():
        autor = [fila.iloc[0]]
        titulo = fila.iloc[1]
        cuerpo = fila.iloc[2]   

        titulo_proc = proceso_contenido(titulo)
        cuerpo_proc = proceso_contenido(cuerpo)

        # Unir las tres listas en una sola lista combinada
        fila_combinada =  autor + titulo_proc + cuerpo_proc

        contenido_final = extraer_noun_chunks(fila_combinada)
        resultados.append(contenido_final)
    return resultados

In [ ]:
# Mostrar los primeros 3 documentos procesados
def prueba_primeros_3_documentos_procesados(corpus):
    for i, documento in enumerate(corpus[:3]):
        print(f"Documento {i+1}:")
        print(" - Palabras:", documento)
        print()

In [ ]:
def expand_term(term):
    related = set()
    for syn in wn.synsets(term):
        for lemma in syn.lemmas():
            word = lemma.name().replace('_', ' ').lower()
            if word != term:
                related.add(word)
    return related

def sinonimos_con_mismo_tfidf(palabra, tfidf, diccionario):
    sinonimos = expand_term(palabra)
    for p in sinonimos:
        diccionario[p]=tfidf
    return diccionario
def tfidf_por_documentos_con_sinonimos(corpus_normalizado):
    corpus = corpus_normalizado

    # 🔁 Convertimos el corpus a una lista de strings para usar TF-IDF
    texts = [" ".join(doc) for doc in corpus]

    # TF-IDF para todo el corpus
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    X = vectorizer.fit_transform(texts)
    terms = vectorizer.get_feature_names_out()

    tfidfs_de_documentos = []

    # Para cada documento se extraen sus palabras y valor tfidf
    for idx, doc in enumerate(corpus):

        submatrix_ = X[idx]  
        tfidf_values = submatrix_.toarray().flatten()  
        tfidf_dict = {}

        # Filtrar solo los términos que están en el documento original
        doc_terms = set(doc)
        for i, term in enumerate(terms):
            if term in doc_terms:
                tfidf_valor = tfidf_values[i]
                tfidf_dict[term] = tfidf_valor
                tfidf_dict = sinonimos_con_mismo_tfidf(term, tfidf_valor, tfidf_dict)


        tfidfs_de_documentos.append(tfidf_dict)
    return tfidfs_de_documentos
        

In [ ]:
def prueba_tfdifs_primeros_3_documentos(lista_diccionarios_tfidfs):
    for idx, d in enumerate(lista_diccionarios_tfidfs[:3]):
        top_terms = sorted(d.items(), key=lambda x: x[1], reverse=True)
        print(f"\n🟩 Documento {idx+1}:")
        print("   ➕ Palabras añadidas por TF-IDF:")
        for term, score in top_terms:
            print(f"      - {term}: {score:.4f}")

In [ ]:
def tfidf_por_documentos_sin_sinonimos(corpus_normalizado):
    corpus = corpus_normalizado

    # 🔁 Convertimos el corpus a una lista de strings para usar TF-IDF
    texts = [" ".join(doc) for doc in corpus]

    # TF-IDF para todo el corpus
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    X = vectorizer.fit_transform(texts)
    terms = vectorizer.get_feature_names_out()

    tfidfs_de_documentos = []

    # Para cada documento se extraen sus palabras y valor tfidf
    for idx, doc in enumerate(corpus):

        submatrix_ = X[idx]  
        tfidf_values = submatrix_.toarray().flatten()  
        tfidf_dict = {}

        # Filtrar solo los términos que están en el documento original
        doc_terms = set(doc)
        for i, term in enumerate(terms):
            if term in doc_terms:
                 tfidf_dict[term] = tfidf_values[i]

        tfidfs_de_documentos.append(tfidf_dict)
    return tfidfs_de_documentos

In [ ]:
# Prueba de lectura 
resultados = lectura_normalizada_corpus()
prueba_primeros_3_documentos_procesados(resultados)

In [ ]:
# Prueba de tfidfs
lista_diccionarios_tfidf = tfidf_por_documentos_sin_sinonimos(resultados)
prueba_tfdifs_primeros_3_documentos(lista_diccionarios_tfidf)

In [ ]:
# Prueba de tfidfs con sinonimos
lista_diccionarios_tfidf_sinonimos = tfidf_por_documentos_con_sinonimos(resultados)
prueba_tfdifs_primeros_3_documentos(lista_diccionarios_tfidf_sinonimos)
